# Merge Youbemom Databases
This notebook first finds and scrapes missing posts then merges the 50 youbemomTables databases into one sqlite database

## Data Sources
- database/youbemom-all/youbemomTables-[01 to 50].db (scraped with group_N.py scripts generated with gen_scripts.py)
- NOTE: 1.0-Create_Data-Scrape_Forum is used for scraping individual subforums. Data used here was scrapped via permalink

## Changes
- 2020-12-08: Created
- 2020-12-09: Added merge databases youbemomTables-[01-40]
- 2020-12-12: Added merge youbemomTables

## Database Structure
- threads
 - id: automatically assigned
 - url: url of top post
 - subforum: subforum of post
 - dne: post does not exist
- posts
 - id: automatically assigned
 - family_id: thread->id
 - message_id: the unique id of the message from the html
 - parent_id: id of post this post is responding to, 0 if top post
 - date_recorded: date the data is fetched
 - date_created: date the data was created
 - title: title of the post
 - body: body of the post
 - subforum: subforum of post
 - deleted: has post been deleted

## TODO
-

## Imports

In [1]:
import sqlite3
from pathlib import Path
from youbemom import loop_list_links, create_connection, set_up_db
import pandas as pd

## Functions

For formatting strings

In [2]:
def pad(n):
    """ pad an int if less than 10 
    :param n: input int
    :return: string padded with 0 if less than 10
    """
    if n < 10:
        return "0" + str(n)
    else:
        return str(n)

In [3]:
def permalink_n(url):
    """ extract permalink number from url
    :param url: url string of permalink
    :return: int of permalink number
    """
    return int(url[17:])

For testing

In [4]:
def check_ids_for_missing(cur):
    """ checks for missing ids and scrapes them 
    :param cur: database cursor
    """
    cur.execute(ids_sql)
    ids = list(cur.fetchall())
    ids = [i[0] for i in ids]
    if ids:
        print("got ids for db-{}".format(n))
        missing_ids = find_missing(ids)
        if len(missing_ids) > 0:
            print(missing_ids)
            cur.execute(min_permalink_sql)
            min_permalink = cur.fetchone()
            if min_permalink:
                min_permalink = permalink_n(min_permalink[0])
                loop_list_links(conn, path_db, missing_ids, min_permalink)

In [5]:
def find_missing(number_list):
    """ extracts all missing ints in the range from the first value
        on the number list to the last number
    :param number_list: ideally sorted, a list of ints
    :return: list of numbers between first and last not in list
    """
    return [x for x in range(number_list[0], number_list[-1] + 1) if x not in number_list] 

## File Locations

In [6]:
p = Path.cwd()
path_parent = p.parents[0]

## Loop Databases
Find missing permalinks in each database that were skipped due to errors in the initial scrape

In [7]:
min_id_sql = ''' SELECT MIN(family_id) FROM threads '''
max_id_sql = ''' SELECT MAX(family_id) FROM threads '''
min_permalink_sql = ''' SELECT url FROM threads ORDER BY ROWID ASC LIMIT 1 '''
ids_sql = ''' SELECT family_id from threads ORDER BY family_id ASC '''
update_threads_sql = ''' UPDATE threads SET family_id = family_id + {} '''
update_posts_sql = ''' UPDATE posts SET family_id = family_id + {} '''

In [ ]:
last_max_value = 0
for i in range(1, 51):
    n = pad(i)
    fn = "youbemomTables-{}.db".format(n)
    path_db = path_parent / "database" / fn
    path_db = str(path_db)
    conn = create_connection(path_db)
    cur = conn.cursor()
    # don't run more than once
    # for each database
    # won't mess anything up but
    # takes a lot of time
    # check_ids_for_missing(cur)
    cur.execute(min_id_sql)
    min_id = cur.fetchone()
    if min_id:
        print("ids start at {}".format(min_id[0]))
        subtract = 1 - int(min_id[0])
        if subtract < 0: # if ids don't start with 1, subtract min_id + 1 so they start at 1
            cur.execute(update_threads_sql.format(subtract))
            cur.execute(update_posts_sql.format(subtract))
    cur.execute(max_id_sql)
    max_id = cur.fetchone() # must do this before updating values
    cur.execute(update_threads_sql.format(last_max_value))
    cur.execute(update_posts_sql.format(last_max_value))
    if max_id:
        last_max_value += int(max_id[0])
    else:
        last_max_value += 70000
    conn.commit()
    conn.close()

In [ ]:
# for i in range(1, 41):
#     n = pad(i)
#     fn = "youbemomTables-{}.db".format(n)
#     path_db = path_parent / "database" / fn
#     path_db = str(path_db)
#     conn = create_connection(path_db)
#     cur = conn.cursor()
#     print("db{}".format(n))
#     counts_sql = '''
#         SELECT subforum, COUNT(family_id), MIN(date_created), MAX(date_created)
#         FROM posts
#         GROUP BY subforum
#     '''
#     counts = pd.read_sql_query(counts_sql, conn)
#     print(counts.head(16))

## Merge Databases
Merge all the databases into one SQLite DB

### Create new database

In [9]:
db = "youbemom-merged.db"
path_db = path_parent / "database" / db
path_db = str(path_db)
conn = create_connection(path_db)
set_up_db(conn)
cur = conn.cursor()
script = '''
    ATTACH DATABASE "{0}" AS db{1};
    INSERT INTO main.threads (family_id, url, subforum, dne) SELECT family_id, url, subforum, dne FROM db{1}.threads;
    INSERT INTO main.posts (family_id, message_id, parent_id, date_recorded, date_created, title, body, subforum, deleted) SELECT family_id, message_id, parent_id, date_recorded, date_created, title, body, subforum, deleted FROM db{1}.posts;
    DETACH DATABASE db{1};
'''

### Merge numbered databases
Merge the databases in youbemomTables-[01-40] into empty youbemomTables-merged

In [10]:
for i in range(1, 51):
    n = pad(i)
    fn = "youbemomTables-{}.db".format(n)
    path_db = path_parent / "database" / fn
    path_db = str(path_db)
    cur.executescript(script.format(path_db, n))

In [11]:
conn.commit()
conn.close()

### Merge longer-range scraped database
Merge the database of looping through pages and not permalinks using pandas to filter and alter the ids

In [ ]:
# min_date_created_sql = ''' SELECT MIN(date_created) FROM posts '''
# min_group_id_sql = ''' SELECT MIN(family_id) FROM posts WHERE date_created < "{0}" AND subforum = "{1}" '''
# max_group_id_sql = ''' SELECT MAX(family_id) FROM posts WHERE date_created < "{0}" AND subforum = "{1}" '''
# script = '''
#     DELETE FROM posts WHERE date_created >= "{0}" AND subforum = "{1}";
#     DELETE FROM threads WHERE subforum = "{1}" AND family_id NOT IN
#         (SELECT family_id 
#         FROM posts 
#         WHERE family_id is NOT NULL AND subforum = "{1}"
#       );
#     UPDATE threads SET family_id = family_id + {2} WHERE subforum = "{1}";
#     UPDATE posts SET family_id = family_id + {2} where subforum = "{1}";
# '''
# forum_list = ["special-needs", "newborn", "preschool", "elementary", "tween-teen"]

Find max family_id to start and min_date to filter

In [ ]:
# db = "youbemom-merged.db"
# path_db = path_parent / "database" / db
# path_db = str(path_db)
# conn = create_connection(path_db)
# cur = conn.cursor()
# cur.execute(max_id_sql)
# max_id = cur.fetchone()
# max_id = int(max_id[0])
# print(max_id)
# cur.execute(min_date_created_sql)
# min_date = cur.fetchone()
# min_date = min_date[0]
# print(min_date)

Renumber family_id in youbemomTables to match youbemom-merged

In [ ]:
# ybm = "youbemomTables.db"
# path_ybm = path_parent / "database" / ybm
# path_ybm = str(path_ybm)

In [ ]:
# conn = create_connection(path_ybm)
# cur = conn.cursor()
# for subforum in forum_list:
#     print(subforum)
#     cur.execute(min_group_id_sql.format(min_date, subforum))
#     min_group_id = cur.fetchone()
#     min_group_id = int(min_group_id[0])
#     print("min id: {}".format(min_group_id))
#     add = max_id - min_group_id + 1
#     print("add: {}".format(add))
#     cur.executescript(script.format(min_date, subforum, add))
#     cur.execute(max_group_id_sql.format(min_date, subforum))
#     max_id = cur.fetchone()
#     max_id = int(max_id[0])

Merge database

In [ ]:
# db = "youbemom-merged.db"
# path_db = path_parent / "database" / db
# path_db = str(path_db)
# conn = create_connection(path_db)
# set_up_db(conn)
# cur = conn.cursor()
# script = '''
#     ATTACH DATABASE "{}" AS ybm;
#     INSERT INTO main.threads (family_id, url, subforum, dne) SELECT family_id, url, subforum, dne FROM ybm.threads;
#     INSERT INTO main.posts (family_id, message_id, parent_id, date_recorded, date_created, title, body, subforum, deleted) SELECT family_id, message_id, parent_id, date_recorded, date_created, title, body, subforum, deleted FROM ybm.posts;
#     DETACH DATABASE ybm;
# '''

In [ ]:
# cur.executescript(script.format(path_ybm))

In [ ]:
# conn.commit()
# conn.close()